# Load Zarr Image with labels from a public S3 repository, analyze using StarDist and compare results

The notebook shows how to load an IDR image converted into a Zarr file with labels.

The image is referenced in the paper "NesSys: a novel method for accurate nuclear segmentation in 3D" published August 2019 in PLOS Biology: https://doi.org/10.1371/journal.pbio.3000388 and can be viewed online in the [Image Data Resource](https://idr.openmicroscopy.org/webclient/?show=image-6001247).

This original image was converted into zarr format. The analysis results produced by the paper authors were converted into labels and linked to the zarr image which was placed into a public S3 repository.

In this notebook, the converted zarr image is then loaded together with the labels from the S3 storage and analyzed using [StarDist](https://stardist.net/docs/index.html). The StarDist analysis produces a segmentation, which is then viewed side-by-side with the original segmentations produced by paper authors obtained via the loaded labels.

In [1]:
import dask
import dask.array as da
import dask_image.ndfilters
import dask_image.ndmeasure
from omero.gateway import BlitzGateway

import matplotlib.pyplot as plt
%matplotlib inline
from dask.diagnostics import ProgressBar
import numpy
import zarr

The Zarr data is stored separately from the IDR, on an S3 object store.

### IDR image to analyze

In [2]:
image_id = 6001247

### Helper methods to load Zarr Image and Labels from S3

In [3]:
def load_binary_from_s3(id, resolution='0'):
    endpoint_url = 'https://s3.embassy.ebi.ac.uk/'
    root = 'idr/zarr/v0.1/%s.zarr/%s/' % (id, resolution)
    with ProgressBar():
        return numpy.asarray(da.from_zarr(endpoint_url + root))

In [4]:
def load_labels_from_s3(id, resolution='0'):
    endpoint_url = 'https://s3.embassy.ebi.ac.uk/'
    root = 'idr/zarr/v0.1/%s.zarr/labels/%s/' % (id, resolution)
    return da.from_zarr(endpoint_url + root)

## Load the binary image from S3 

In [5]:
data = load_binary_from_s3(image_id)
print(data.shape)

[########################################] | 100% Completed | 20.1s
(1, 2, 257, 210, 253)


## Load the labels
Labels are not on all planes, binary will be loaded when viewing the labels.

In [6]:
labels = load_labels_from_s3(image_id)
print(labels.shape)

(1, 1, 257, 210, 253)


## Load stardist trained model 

In [8]:
from stardist.models import StarDist2D
model_versatile = StarDist2D.from_pretrained('2D_demo')

Found model '2D_demo' for 'StarDist2D'.
5300224/5298331 [==============================] - 2s 0us/step
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.486166, nms_thresh=0.5.


## Prediction based on StarDist model
Normalize the input image

``model.predict_instances`` will

 * predict object probabilities and star-convex polygon distances (see model.predict if you want those)
 * perform non-maximum suppression (with overlap threshold nms_thresh) for polygons above object probability threshold prob_thresh.
 * render all remaining polygon instances in a label image
 * return the label instances image and also the details (coordinates, etc.) of all remaining polygons

In [9]:
from csbdeep.utils import normalize
axis_norm = (0,1)
img = normalize(data[0, 0, :, :, :], 1,99.8, axis=axis_norm)
results = []
for im in img:
    new_labels, details = model_versatile.predict_instances(im)
    results.append(new_labels)

label_slices = numpy.array(results)

## Compare labels
Display the original labels and the labels based on StarDist prediction side-by-side

In [10]:
from ipywidgets import *

def update(z=0):
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(data[0, 0, z, :, :], cmap='gray')
    try:
        plt.imshow(labels[0, 0, z, :, :], cmap='jet', alpha=0.5)
    except Exception:
        print(z)
    plt.subplot(122)
    plt.imshow(data[0, 0, z, :, :], cmap='gray')
    plt.imshow(label_slices[z, :, :], cmap='jet', alpha=0.5)
    plt.tight_layout()
    fig.canvas.flush_events()

interact(update, z= widgets.IntSlider(value=0, min=0, max=data.shape[2]-1, step=1, description="Select Z", continuous_update=False))

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Z', max=256), Output()),…

<function __main__.update(z=0)>

### License
Copyright (C) 2020 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.